# export trials to JSON

In [ ]:
import sys

sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append(
    "/home/webphy/Desktop/dnn_processor/dataset/"
)  # just to enable `dataset`
import dataset

import tensorflow as tf
import keras 
from glob import glob
import json

# load dataset
train_images_rgx = "../../dataset/resized_dataset_texturas_v2/*/*.jpeg"
valid_images_rgx = "../../dataset/resized_dataset_texturas_v2_test/*/*.jpeg"
train_ds, val_ds, nclass = dataset.create_datasets(
    train_images_rgx, valid_images_rgx, 64
)

def count_parameters(model):
    total_params = 0
    for layer in model.trainable_variables:
        layer_params = tf.reduce_prod(layer.shape)
        total_params += layer_params
    return total_params.numpy()


def count_parameters_of_layer(layer):
    total_params = 0
    for l in layer.trainable_variables:
        layer_params = tf.reduce_prod(l.shape)
        total_params += layer_params
    return total_params.numpy()

In [ ]:
exp_id = "exp4_with_aug"

saved_models_root = f"saved_models/{exp_id}/*/"
models_path = glob(saved_models_root)
assert len(models_path) > 0, "nenhum arquivo listado"

trials = {}
keras.utils.disable_interactive_logging()
for p in models_path:
    print(p)
    trial_id = p.split("model_t")[1][:-1]

    # load model and weights
    model = keras.models.load_model(p)
    model.summary()
    model.load_weights(p + "/weights/")

    # evaluate
    val_score = model.evaluate(val_ds, verbose=1)

    # if the val acc is less than 90% the model is discarded
    if val_score[1] < 0.9:
        continue

    train_score = model.evaluate(train_ds, verbose=1)

    model_num_params = count_parameters(model)
    dense_num_params = count_parameters_of_layer(model.layers[-1])

    trials[int(trial_id)] = {
        "trial_id": trial_id,
        "train_loss": train_score[0],
        "train_accuracy": train_score[1],
        "val_loss": val_score[0],
        "val_accuracy": val_score[1],
        "num_params": model_num_params,
        "dense_num_params": dense_num_params,
        "model_config": model.get_config(),
    }


with open(f"{exp_id}.json", "w") as f:
    json.dump(trials, f)

In [ ]:
# Create a list of tuples with (num_params, object)
ordered_list = [(trials[key]["num_params"], trials[key]) for key in trials]

# Sort the list based on num_params
ordered_list.sort(key=lambda x: x[0])

# Extract the objects from the sorted list
ordered_objects = [obj for _, obj in ordered_list]

with open(f"{exp_id}_ordered_objects.json", "w") as f:
    json.dump(ordered_objects, f)